### Import bibliotek

In [1]:
import json
import requests
import os
from bs4 import BeautifulSoup

### Pobieranie ze strony Ceneo.pl opinii o konkretnym produkcie

In [ ]:
with open("./headers.json","r") as jf:
    headers = json.load(jf)

In [ ]:
product_id = "84514582"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_reviews = []
while next_page:
    response = response.get(next_page)
    print(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        reviews = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(reviews))
    for review in reviews:
        try:
            single_review = {
                "review_id": review['data-entry-id'],
                "author": review.select_one("span.user-post__author-name").text.strip(),
                "recomendation": review.select_one("span.user-post__author-recomendation > em").text.strip(),
                "stars": review.select_one("span.user-post__score-count").text.strip(),
                "pros": [p.text.strip() for p in review.select("div.review-feature__item--positive")],
                "cons": [p.text.strip() for p in review.select("div.review-feature__item--negative")],
                "content" : review.select_one("div.user-post__text").text.strip(),
                "likes" : review.select_one("button.vote-yes > span").text.strip(),
                "dislikes" : review.select_one("button.vote-no > span").text.strip(),
                "publish_date" : review.select_one("span.user-post__published > time:nth-child(1)['datetime']").strip(),
                "purchase_date" : review.select_one("span.user-post__published > time:nth-child(2)['datetime']").strip(),
            }
            all_reviews.append(single_review)

        except (AttributeError, TypeError):
            pass
    try:
        next_page = "htttps://www.ceneo.pl" + page_dom.select_one("a.pagination_next")["href"]
    except TypeError:
        next_page = None
print(all_reviews)

    

Zapisanie wszystkich opinii w bazie danych

In [2]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")
    

In [ ]:
with open(f"./opinions/{product_id}.json","w",encoding="UTF-8") as jf:
    json.dump(all_reviews, jf, indent=4, ensure_ascii=False)